In [ ]:
!pip install fedartml
!pip install -q flwr[simulation]

In [ ]:
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve

from io import BytesIO
import requests

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.optimizers import SGD

from flwr.simulation.ray_transport.utils import enable_tf_gpu_growth
enable_tf_gpu_growth()

import time
import pickle

import pandas as pd

import matplotlib.pyplot as plt

# Garbage Collector - use it like gc.collect()
import gc

# Custom Callback To Include in Callbacks List At Training Time
class GarbageCollectorCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()

from fedartml import InteractivePlots, SplitAsFederatedData

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

import flwr as fl
from typing import List, Tuple, Dict, Optional
from flwr.common import Metrics

In [ ]:
# Define function to test a model and retrieve classification metrics
def test_model(model, X_test, Y_test):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = False)
    logits = model.predict(X_test, batch_size=8, verbose=2, callbacks=[GarbageCollectorCallback()])
    y_pred = tf.argmax(logits, axis=1)
    loss = cce(Y_test, logits).numpy()
    acc = accuracy_score(y_pred, Y_test)
    pre = precision_score(y_pred, Y_test, average='weighted',zero_division = 0)
    rec = recall_score(y_pred, Y_test, average='weighted',zero_division = 0)
    f1s = f1_score(y_pred, Y_test, average='weighted',zero_division = 0)

    return loss, acc, pre, rec, f1s

# Define function to convert from SplitAsFederatedData function output (FedArtML) to Flower (list) format
def from_FedArtML_to_Flower_format(clients_dict):
  # initialize list that contains clients (features and labels) to extract later from client_fn in Flower
  list_x_train = []
  list_y_train = []

  # Get the name of the clients from the dictionary
  client_names = list(clients_dict.keys())

  # Iterate over each client
  for client in client_names:
    # Get data from each client
    each_client_train=np.array(clients_dict[client],dtype=object)

    # Extract features for each client
    feat=[]
    x_tra=np.array(each_client_train[:, 0])
    for row in x_tra:
      feat.append(row)
    feat=np.array(feat)

    # Extract labels from each client
    y_tra=np.array(each_client_train[:, 1])

    # Append in list features and labels to extract later from client_fn in Flower
    list_x_train.append(feat)
    list_y_train.append(y_tra)

  return list_x_train, list_y_train

def get_model():
    """Constructs a simple model architecture suitable for MNIST."""
    model = Sequential([
        tf.keras.layers.Conv2D(6, kernel_size=(5, 5), activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Conv2D(16, kernel_size=(5, 5), activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(120, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(84, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer=SGD(learning_rate = 0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

class FlowerClient(fl.client.NumPyClient):
    def __init__(self, model, x_train, y_train, x_test, y_test, epochs_client) -> None:
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_test, self.y_test = x_test, y_test
        self.epochs_client = epochs_client

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=self.epochs_client, verbose=2, batch_size = 64, callbacks=[GarbageCollectorCallback()])
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        self.model.compile(optimizer='SGD', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
        loss, acc = self.model.evaluate(self.x_test, self.y_test, verbose=2)
        return loss, len(self.x_test), {"accuracy": acc}

def plot_metric_from_history(
    hist: None,
    save_plot_path: None,
    metric_type: None,
    metric: None,
) -> None:

    metric_dict = (
        hist.metrics_centralized
        if metric_type == "centralized"
        else hist.metrics_distributed
    )
    rounds, values = zip(*metric_dict[metric])
    # plt.plot(np.asarray(rounds), np.asarray(values), label="FedAvg")
    plt.plot(np.asarray(rounds), np.asarray(values), color=colors[5], linewidth=5, label='Test')
    plt.legend(fontsize=45)
    plt.xlabel('Communication round', fontsize=40)
    plt.ylabel(metric, fontsize=50)
    plt.title(metric, fontsize=60)
    plt.xticks(fontsize=30)
    plt.yticks(fontsize=30)
    # plt.ylim(min(min(min(commun_metrics))) - 0.05, max(max(max(commun_metrics))) + 0.05)
    plt.ylim(0, 1)
    # plt.savefig(Path(save_plot_path) / Path(f"{metric_type}_metrics{suffix}.png"))
    # plt.close()

def retrieve_global_metrics(
    hist: None,
    metric_type: None,
    metric: None,
    best_metric: None,
) -> None:
    """Function to plot from Flower server History.
    Parameters
    ----------
    hist : History
        Object containing evaluation for all rounds.
    metric_type : Literal["centralized", "distributed"]
        Type of metric to retrieve.
    metric : Literal["accuracy","precision","recall","f1score"]
        Metric to retrieve.
    """
    metric_dict = (
        hist.metrics_centralized
        if metric_type == "centralized"
        else hist.metrics_distributed
    )
    rounds, values = zip(*metric_dict[metric])
    if best_metric:
      metric_return = max(values)
    else:
      metric_return = values[-1]
    return metric_return

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
from collections import Counter
import math

def get_hist(y_train):
    counts = Counter(y_train)
    counts = list(counts.values())
    return counts

def euclidean_distance(hist1, hist2):
    sum_hist = np.add(hist1, hist2)
    return np.sum(np.abs(np.subtract(sum_hist, np.sum(sum_hist)/len(sum_hist))))

def find_fit_histogram(histograms_info, target_histogram):
    histograms = [histogram for histogram, cid in histograms_info]
    cid = [cid for histogram, cid in histograms_info]
    distances = [euclidean_distance(hist, target_histogram) for hist in histograms]
    return cid[np.argmin(distances)]

def calculate_entropy(hist):
        entropy = 0.0
        counts = [0 if value is None else value for value in hist]
        for value in counts:
            entropy += -value/sum(counts) * math.log(value/sum(counts), 10) if value != 0 else 0
        return entropy

In [ ]:
from logging import WARNING
from typing import Callable, Dict, List, Optional, Tuple, Union
from functools import reduce

from flwr.common import (
    FitRes,
    FitIns,
    MetricsAggregationFn,
    NDArrays,
    Parameters,
    Scalar,
    ndarrays_to_parameters,
    parameters_to_ndarrays,
)
from flwr.common.logger import log
from flwr.server.client_manager import ClientManager
from flwr.server.client_proxy import ClientProxy
import random
from flwr.server.strategy.fedavg import FedAvg
from flwr.server.strategy.aggregate import aggregate, aggregate_inplace, weighted_loss_avg, aggregate_median
import random

class FedTransfer(FedAvg):
    def __init__(
        self,
        *,
        fraction_fit: float = 1.0,
        fraction_evaluate: float = 1.0,
        min_fit_clients: int = 2,
        min_evaluate_clients: int = 2,
        min_available_clients: int = 2,
        evaluate_fn: Optional[
            Callable[
                [int, NDArrays, Dict[str, Scalar]],
                Optional[Tuple[float, Dict[str, Scalar]]],
            ]
        ] = None,
        on_fit_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        on_evaluate_config_fn: Optional[Callable[[int], Dict[str, Scalar]]] = None,
        accept_failures: bool = True,
        initial_parameters: Optional[Parameters] = None,
        fit_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        evaluate_metrics_aggregation_fn: Optional[MetricsAggregationFn] = None,
        inplace: bool = True,
    ) -> None:
        super().__init__()
        self.fraction_fit = fraction_fit
        self.fraction_evaluate = fraction_evaluate
        self.min_fit_clients = min_fit_clients
        self.min_evaluate_clients = min_evaluate_clients
        self.min_available_clients = min_available_clients
        self.evaluate_fn = evaluate_fn
        self.on_fit_config_fn = on_fit_config_fn
        self.on_evaluate_config_fn = on_evaluate_config_fn
        self.accept_failures = accept_failures
        self.initial_parameters = initial_parameters
        self.fit_metrics_aggregation_fn = fit_metrics_aggregation_fn
        self.evaluate_metrics_aggregation_fn = evaluate_metrics_aggregation_fn
        self.inplace = inplace
        self.parameters_results = []
        self.current_hists = None

    def configure_fit(
          self, server_round: int, parameters: Parameters, client_manager: ClientManager
      ) -> List[Tuple[ClientProxy, FitIns]]:
          """Configure the next round of training."""
          config = {}
          if self.on_fit_config_fn is not None:
              # Custom fit config function provided
              config = self.on_fit_config_fn(server_round)

          sample_size, min_num_clients = self.num_fit_clients(
              client_manager.num_available()
          )
          clients, self.entropy = client_manager.sample(
              num_clients=sample_size, min_num_clients=min_num_clients
          )
          if len(self.parameters_results) == 0:
              self.parameters_results = [parameters for _ in clients]

          fit_ans = [FitIns(params, config) for params in self.parameters_results]
          return [(clients[i], fit_ans[i]) for i in range(len(clients))]

    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[Union[Tuple[ClientProxy, FitRes], BaseException]],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        if not results:
            return None, {}
        # Do not aggregate if there are failures and failures are not accepted
        if not self.accept_failures and failures:
            return None, {}

        # Convert results
        weights_results = [
            (parameters_to_ndarrays(fit_res.parameters), fit_res.num_examples)
            for _, fit_res in results
        ]

        parameters_aggregated = ndarrays_to_parameters(
            aggregate_median(weights_results)
        )

        if self.entropy >= 0.8:
            self.parameters_results = [parameters_aggregated for _, fit_res in results]
            print("reset!")
        else:
            self.parameters_results = [fit_res.parameters for _, fit_res in results]


        # Aggregate custom metrics if aggregation fn was provided
        metrics_aggregated = {}
        if self.fit_metrics_aggregation_fn:
            fit_metrics = [(res.num_examples, res.metrics) for _, res in results]
            metrics_aggregated = self.fit_metrics_aggregation_fn(fit_metrics)
        elif server_round == 1:  # Only log this warning once
            log(WARNING, "No fit_metrics_aggregation_fn provided")

        return parameters_aggregated, metrics_aggregated

In [ ]:
import flwr as fl
from flwr.server.client_manager import ClientManager
import threading
from abc import ABC, abstractmethod
from logging import INFO
from flwr.common.logger import log
from typing import Dict, List, Optional
import random
from flwr.server.client_proxy import ClientProxy
from flwr.server.criterion import Criterion

class SimpleClientManager(ClientManager):

    def __init__(self, hists) -> None:
        self.clients: Dict[str, ClientProxy] = {}
        self._cv = threading.Condition()
        self.hists = hists
        self.busy = [0 for _ in range(len(hists))]
        self.previous_clients = None
        self.reset_round = False

    def __len__(self) -> int:
        return len(self.clients)

    def num_available(self) -> int:
        return len(self)

    def wait_for(self, num_clients: int, timeout: int = 86400) -> bool:
        with self._cv:
            return self._cv.wait_for(
                lambda: len(self.clients) >= num_clients, timeout=timeout
            )

    def register(self, client: ClientProxy) -> bool:
        if client.cid in self.clients:
            return False

        self.clients[client.cid] = client
        with self._cv:
            self._cv.notify_all()

        return True

    def unregister(self, client: ClientProxy) -> None:
        if client.cid in self.clients:
            del self.clients[client.cid]

            with self._cv:
                self._cv.notify_all()

    def all(self) -> Dict[str, ClientProxy]:
        return self.clients

    def sample(
        self,
        num_clients: int,
        min_num_clients: Optional[int] = None,
        criterion: Optional[Criterion] = None,
    )-> List[ClientProxy]:
        if min_num_clients is None:
            min_num_clients = num_clients
        self.wait_for(min_num_clients)
        # Sample clients which meet the criterion
        available_cids = list(self.clients)
        if criterion is not None:
            available_cids = [
                cid for cid in available_cids if criterion.select(self.clients[cid])
            ]
        if num_clients == 1:
            sampled_cids = random.sample(available_cids, num_clients)
            return [self.clients[cid] for cid in sampled_cids]

        if self.previous_clients == None or self.reset_round == True:
            available_cids = list(self.clients)
            sampled_cids = random.sample(available_cids, num_clients)
            self.model_hist = [self.hists[int(cid)] for cid in sampled_cids]
            global_hist = np.sum(self.model_hist, axis =0)
            entropy = calculate_entropy(global_hist)
            if entropy >= 0.8:
                self.reset_round = True
                self.model_hist = [global_hist for _ in range(len(sampled_cids))]
            for cid in sampled_cids:
                self.busy[int(cid)] = 1
            return [(self.clients[cid]) for cid in sampled_cids], entropy
        else:
            available_cids = [str(cid) for cid, _ in enumarate(self.busy) if self.busy[cid] == 0]
            available_hists = [(hists[int(cid)],cid) for cid in available_cids]
            for parameters, cid in self.previous_clients:
                self.busy[int(cid)] = 0
            sampled_cids = []
            for cid in self.previous_clients:
                  hist = self.model_hist[int(cid)]
                  fit_hist = find_fit_histogram(available_hists, hist)
                  sampled_cids.append(fit_hist)
                  for h in available_hists:
                      if h[1] == cid:
                          available_hists.remove(h)
                          break;
            self.model_hist = [np.add(current_hists, hists[int(cid)]) for current_hists, cid in zip(self.model_hist, sampled_cids)]
            global_hist = np.sum(self.model_hist, axis =0)
            entropy = calculate_entropy(global_hist)
            if entropy >= 0.8:
                self.reset_round = True
                self.model_hist = [global_hist for _ in range(len(sampled_cids))]
            for cid in sampled_cids:
                self.busy[int(cid)] = 1
            return [(self.clients[cid]) for cid in sampled_cids], entropy

In [ ]:
# Define random state for reproducibility
random_state = 0

# Define colors to use in plots
colors = ["#00cfcc","#e6013b","#007f88","#00cccd","#69e0da","darkblue","#FFFFFF"]

# Define number of local nodes to be used
local_nodes_glob = 50

# Define percentage of noniid to be used
Percent_noniid = 90

Alpha = 0.1

In [ ]:
from tensorflow.keras.datasets import mnist, cifar10
from tensorflow.keras.utils import to_categorical

# Define random state for reproducibility
random_state = 2

# Tải dữ liệu MNIST
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28, 28, 1)).astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1)).astype('float32') / 255
# Instantiate a SplitAsFederatedData object
my_federater = SplitAsFederatedData(random_state = random_state)

# Get federated dataset from centralized dataset
clients_glob_dic, list_ids_sampled_dic, miss_class_per_node, distances = my_federater.create_clients(image_list = train_images, label_list = train_labels,
                                                             num_clients = local_nodes_glob, prefix_cli='client', method = "dirichlet", alpha = Alpha)

clients_glob = clients_glob_dic['with_class_completion']
list_ids_sampled = list_ids_sampled_dic['with_class_completion']

list_x_train, list_y_train = from_FedArtML_to_Flower_format(clients_dict=clients_glob)


11490434/11490434 [==============================] - 0s 0us/step


In [ ]:
# The `evaluate` function will be by Flower called after every round
def evaluate_DNN_CL(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = get_model()
    net.set_weights(parameters) # Update model with the latest parameters
    loss, accuracy, precision, recall, f1score  = test_model(net, test_images, test_labels)
    print(f"@@@@@@ Server-side evaluation loss {loss} / accuracy {accuracy} / f1score {f1score} @@@@@@")
    return loss, {"accuracy": accuracy,"precision": precision,"recall": recall,"f1score": f1score}

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Define number of epochs per local node
epochs = 1

# Define number of communication rounds
comms_round = 100

# Define dunction to pass to each local node (client)
def client_fn(cid: str) -> fl.client.Client:
    # Define model
    model = get_model()

    # Load data partition of each client ID (cid)
    x_train_cid = np.array(list_x_train[int(cid)],dtype=float)
    y_train_cid = np.array(list_y_train[int(cid)],dtype=float)

    # Define test data (taken from the centralized data to compare performance of CL and FL)
    x_test_cid = np.array(test_images)
    y_test_cid = np.array(test_labels)

    # Create and return client
    return FlowerClient(model, x_train_cid, y_train_cid, x_test_cid, y_test_cid, epochs)

# Create Federated strategy
strategy= FedTransfer(
        fraction_fit=0.2,  # Sample 100% of available clients for training
        fraction_evaluate=0.0,  # Sample 50% of available clients for evaluation
        min_fit_clients=10,
        min_evaluate_clients = 0,
        min_available_clients = 50,
        evaluate_fn=evaluate_DNN_CL,
)
hists = [get_hist(y_train) for y_train in list_y_train]

start_time = time.time()
# Start simulation (training FL)
commun_metrics_history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=local_nodes_glob,
    config=fl.server.ServerConfig(num_rounds=comms_round),
    strategy=strategy,
    client_manager = SimpleClientManager(hists),
    actor_kwargs={
        "on_actor_init_fn": enable_tf_gpu_growth # <-- To be executed upon actor init.
    },
)

# Get total training time
training_time = time.time() - start_time

# Unmask metrics
global_acc_test = retrieve_global_metrics(commun_metrics_history,"centralized","accuracy",True)

global_pre_test = retrieve_global_metrics(commun_metrics_history,"centralized","precision",True)

global_rec_test = retrieve_global_metrics(commun_metrics_history,"centralized","recall",True)

global_f1s_test = retrieve_global_metrics(commun_metrics_history,"centralized","f1score",True)

print("\n\nFINAL RESULTS: ===========================================================================================================================================================================================")
print('Test: commun_round: {} | global_acc: {:} | global_pre: {} | global_rec: {} | global_f1s: {}'.format(comms_round, global_acc_test, global_pre_test, global_rec_test, global_f1s_test))
print("Training time: %s seconds" % (training_time))

INFO flwr 2024-03-20 17:58:51,570 | app.py:178 | Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=100, round_timeout=None)
2024-03-20 17:58:56,920	INFO worker.py:1621 -- Started a local Ray instance.
INFO flwr 2024-03-20 17:59:00,508 | app.py:213 | Flower VCE: Ray initialized with resources: {'object_store_memory': 3848895283.0, 'memory': 7697790567.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'object_store_memory': 3848895283.0, 'memory': 7697790567.0, 'node:172.28.0.12': 1.0, 'node:__internal_head__': 1.0, 'CPU': 2.0}
INFO flwr 2024-03-20 17:59:00,515 | app.py:219 | Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO:flwr:Optimize your simulation with Flower VCE: https://flower.dev/docs/framework/how-to-run-simulations.html
INFO flwr 2024-03-20

1250/1250 - 4s - 4s/epoch - 3ms/step


INFO flwr 2024-03-20 17:59:13,598 | server.py:94 | initial parameters (loss, other metrics): 2.3025248, {'accuracy': 0.1077, 'precision': 0.34618078584998274, 'recall': 0.1077, 'f1score': 0.1439857458369794}
INFO:flwr:initial parameters (loss, other metrics): 2.3025248, {'accuracy': 0.1077, 'precision': 0.34618078584998274, 'recall': 0.1077, 'f1score': 0.1439857458369794}
INFO flwr 2024-03-20 17:59:13,602 | server.py:104 | FL starting
INFO:flwr:FL starting
DEBUG flwr 2024-03-20 17:59:13,606 | server.py:222 | fit_round 1: strategy sampled 10 clients (out of 50)
DEBUG:flwr:fit_round 1: strategy sampled 10 clients (out of 50)


@@@@@@ Server-side evaluation loss 2.3025248050689697 / accuracy 0.1077 / f1score 0.1439857458369794 @@@@@@


(DefaultActor pid=51300) /usr/local/lib/python3.10/dist-packages/flwr/simulation/ray_transport/ray_actor.py:72: DeprecationWarning:  Ensure your client is of type `flwr.client.Client`. Please convert it using the `.to_client()` method before returning it in the `client_fn` you pass to `start_simulation`. We have applied this conversion on your behalf. Not returning a `Client` might trigger an error in future versions of Flower.
(DefaultActor pid=51300)   client = check_clientfn_returns_client(client_fn(cid))


...

@@@@@@ Server-side evaluation loss 0.7010301947593689 / accuracy 0.798 / f1score 0.8040623602836524 @@@@@@
(DefaultActor pid=51301) 14/14 - 4s - loss: 0.6521 - accuracy: 0.7986 - 4s/epoch - 259ms/step [repeated 2x across cluster]
(DefaultActor pid=51301) 11/11 - 4s - loss: 0.8166 - accuracy: 0.7762 - 4s/epoch - 369ms/step [repeated 4x across cluster]
(DefaultActor pid=51300) 4/4 - 4s - loss: 0.5473 - accuracy: 0.8744 - 4s/epoch - 911ms/step [repeated 3x across cluster]


DEBUG flwr 2024-03-20 18:36:37,913 | server.py:236 | fit_round 99 received 10 results and 0 failures
DEBUG:flwr:fit_round 99 received 10 results and 0 failures


1250/1250 - 3s - 3s/epoch - 2ms/step


INFO flwr 2024-03-20 18:36:43,329 | server.py:125 | fit progress: (99, 0.70454854, {'accuracy': 0.8147, 'precision': 0.85304744041136, 'recall': 0.8147, 'f1score': 0.8194200155935459}, 2249.72336026)
INFO:flwr:fit progress: (99, 0.70454854, {'accuracy': 0.8147, 'precision': 0.85304744041136, 'recall': 0.8147, 'f1score': 0.8194200155935459}, 2249.72336026)
INFO flwr 2024-03-20 18:36:43,336 | server.py:171 | evaluate_round 99: no clients selected, cancel
INFO:flwr:evaluate_round 99: no clients selected, cancel
DEBUG flwr 2024-03-20 18:36:43,340 | server.py:222 | fit_round 100: strategy sampled 10 clients (out of 50)
DEBUG:flwr:fit_round 100: strategy sampled 10 clients (out of 50)


@@@@@@ Server-side evaluation loss 0.7045485377311707 / accuracy 0.8147 / f1score 0.8194200155935459 @@@@@@
(DefaultActor pid=51300) 5/5 - 3s - loss: 0.7116 - accuracy: 0.7818 - 3s/epoch - 681ms/step [repeated 3x across cluster]
(DefaultActor pid=51300) 9/9 - 4s - loss: 0.7603 - accuracy: 0.7862 - 4s/epoch - 452ms/step [repeated 4x across cluster]


In [ ]:
# Define metrics to plot
metrics_show = ["accuracy","precision","recall","f1score"]

# Define dimensions for plot
f, axs = plt.subplots(1,len(metrics_show),figsize=(70,15))

# Loop over the communication round history and metrics
for i in range(len(metrics_show)):
  plt.subplot(1, len(metrics_show), i + 1)
  plot_metric_from_history(commun_metrics_history,"any","centralized",metrics_show[i])